# Initialize

In [1]:
#import graphlearning.active_learning as al
import graphlearning as gl
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import math
import time
import timeit
import os, glob

#SKLearn imports
import sklearn.datasets as datasets
from sklearn.manifold import TSNE, SpectralEmbedding
from sklearn.cluster._kmeans import _kmeans_plusplus
from sklearn.utils import check_random_state
from sklearn.preprocessing import normalize
from sklearn.datasets import make_blobs

#Scipy imports
from scipy.linalg import sqrtm
from scipy.sparse import coo_matrix, csr_matrix
from scipy.sparse.linalg import eigs
import scipy.sparse as sparse
from scipy.optimize import fsolve
from scipy import spatial

In [2]:
#torch imports
import torchvision
import graphlearning.active_learning

In [3]:
import utils
#import utils_sar
import batch_active_learning as bal

In [4]:
from importlib import reload
reload(bal)
#reload(utils_sar)

<module 'batch_active_learning' from '/Users/jameschapman/Documents/GitHub/SAR_BAL/batch_active_learning.py'>

In [5]:
from torchvision.models.feature_extraction import create_feature_extractor

# Test batch active learning

## MSTAR dataset

In [6]:
hdr, fields, mag, phase = utils.load_MSTAR()
train_mask, test_mask, _ = utils.train_test_split(hdr,1)
labels, target_names = utils.targets_to_labels(hdr)

vae_fname = 'SAR10_CNNVAE'
model_fpath = os.path.join("..", "models", vae_fname + ".pt")
assert os.path.exists(model_fpath)  

dataset, metric = vae_fname.split("_")
train_idx_all = np.where(train_mask)[0]

X = utils.encodeMSTAR(model_fpath, use_phase=True)
try:
    knn_data = gl.weightmatrix.load_knn_data(dataset,metric=metric)
except:
    knn_data = gl.weightmatrix.knnsearch(X,50,similarity='angular',dataset=dataset,metric=metric)

knn_num = 20
W = gl.weightmatrix.knn(None,knn_num,knn_data=knn_data)
G = gl.graph(W)

initial = gl.trainsets.generate(labels, rate=1).tolist()

FileNotFoundError: [Errno 2] No such file or directory: '/data/MSTAR/SAR10a.npz'

In [ ]:
method = 'Laplace'
acq_fun = 'uc'

In [ ]:
density_radius_param = .25
coreset = coreset_dijkstras(G, rad = 0.2, DEBUGGING=False, data = X, initial=initial,
                                density_info = (True, density_radius_param, 1))
print("Coreset Size = {}\t Percent of data = {}%".format(len(coreset), round(100 * len(coreset) / len(X), 2)))
print("Coreset = ", coreset)

### same acq_fun, different methods

In [ ]:
# acq_fun_list = ['uc', 'vopt', 'mc', 'mcvopt']
acq_fun_list = ['uc']
max_new_samples = 1000
batchsize=15

for acq_fun in acq_fun_list:
  L_time = []
  L_num_labels = []
  L_acc = []
  L_names = []

  # al_mtd = 'global_max'
  # start = timeit.default_timer() 
  # _, list_num_labels, list_acc = bal.coreset_run_experiment(X, labels, W, coreset, num_iter=max_new_samples, method='Laplace',
  #                          display=False, use_prior=False, al_mtd=al_mtd, debug=False,
  #                          acq_fun=acq_fun, knn_data=knn_data, mtd_para=None,
  #                          savefig=False, savefig_folder='../BAL_figures', batchsize=batchsize,
  #                          dist_metric='angular', q=5, thresholding=0, randseed=0)
  # stop = timeit.default_timer()
  # L_time.append(stop - start)
  # L_num_labels.append(list_num_labels)
  # L_acc.append(list_acc)
  # L_names.append(al_mtd)

  num_iter = int(max_new_samples/batchsize)

  al_mtd = 'local_max'
  start = timeit.default_timer() 
  _, list_num_labels, list_acc = bal.coreset_run_experiment(X, labels, W, coreset, num_iter=num_iter, method='Laplace',
                           display=False, use_prior=False, al_mtd=al_mtd, debug=False,
                           acq_fun=acq_fun, knn_data=knn_data, mtd_para=None,
                           savefig=False, savefig_folder='../BAL_figures', batchsize=batchsize,
                           dist_metric='angular', q=5, thresholding=0, randseed=0)
  stop = timeit.default_timer()
  L_time.append(stop - start)
  L_num_labels.append(list_num_labels)
  L_acc.append(list_acc)
  L_names.append(al_mtd)

  al_mtd = 'rs_kmeans'
  mtd_para = (0.05, 'dijkstra', 1.0, 0.1, 0.2, False, 200, False)
  start = timeit.default_timer() 
  _, list_num_labels, list_acc = bal.coreset_run_experiment(X, labels, W, coreset, num_iter=num_iter, method='Laplace',
                           display=False, use_prior=False, al_mtd=al_mtd, debug=False,
                           acq_fun=acq_fun, knn_data=knn_data, mtd_para=mtd_para,
                           savefig=False, savefig_folder='../BAL_figures', batchsize=batchsize,
                           dist_metric='angular', q=5, thresholding=0, randseed=0)
  stop = timeit.default_timer()
  L_time.append(stop - start)
  L_num_labels.append(list_num_labels)
  L_acc.append(list_acc)
  L_names.append(al_mtd)

  al_mtd = 'acq_sample'
  start = timeit.default_timer() 
  _, list_num_labels, list_acc = bal.coreset_run_experiment(X, labels, W, coreset, num_iter=num_iter, method='Laplace',
                           display=False, use_prior=False, al_mtd=al_mtd, debug=False,
                           acq_fun=acq_fun, knn_data=knn_data, mtd_para=None,
                           savefig=False, savefig_folder='../BAL_figures', batchsize=batchsize,
                           dist_metric='angular', q=5, thresholding=0, randseed=0)
  stop = timeit.default_timer()
  L_time.append(stop - start)
  L_num_labels.append(list_num_labels)
  L_acc.append(list_acc)
  L_names.append(al_mtd)

  al_mtd = 'rs_kmeans'
  cw = lambda x,y: ( x / (x + y) ) ** 10
  mtd_para = (0.05, 'dijkstra', 1.0, 0.1, 0.2, cw, 200, False)
  start = timeit.default_timer() 
  _, list_num_labels, list_acc = bal.coreset_run_experiment(X, labels, W, coreset, num_iter=num_iter, method='Laplace',
                           display=False, use_prior=False, al_mtd=al_mtd, debug=False,
                           acq_fun=acq_fun, knn_data=knn_data, mtd_para=None,
                           savefig=False, savefig_folder='../BAL_figures', batchsize=batchsize,
                           dist_metric='angular', q=5, thresholding=0, randseed=0)
  stop = timeit.default_timer()
  L_time.append(stop - start)
  L_num_labels.append(list_num_labels)
  L_acc.append(list_acc)
  L_names.append(al_mtd + '_c')
    
  for i in range(len(L_acc)):
    plt.plot(L_num_labels[i], L_acc[i], label = L_names[i])

  plt.legend()

  title_str = 'Coreset size: ' + str(len(coreset)) + '; GL method: ' + method + '; Al function: ' + acq_fun + '\n'
  title_str += 'MSTAR CNNVAE: KNN: ' + str(knn_num) + '; Total num data: ' + str(X.shape[0])

  plt.title(title_str)
  plt.savefig(os.path.join('../BAL_figures', 'MSTAR_'+acq_fun+'_diff_mtd.png'))
  plt.show()

### same method, different acq_funs

In [ ]:
acq_fun_list = ['uc', 'vopt', 'mc', 'mcvopt']
max_new_samples = 1000
batchsize=15

L_time = []
L_num_labels = []
L_acc = []
L_names = []

for acq_fun in acq_fun_list:

  al_mtd = 'global_max'
  print(acq_fun, al_mtd)
  start = timeit.default_timer() 
  _, list_num_labels, list_acc = bal.coreset_run_experiment(X, labels, W, coreset, num_iter=max_new_samples, method='Laplace',
                           display=False, use_prior=False, al_mtd=al_mtd, debug=False,
                           acq_fun=acq_fun, knn_data=knn_data, mtd_para=None,
                           savefig=False, savefig_folder='../BAL_figures', batchsize=batchsize,
                           dist_metric='angular', q=5, thresholding=0, randseed=0)
  stop = timeit.default_timer()
  L_time.append(stop - start)
  L_num_labels.append(list_num_labels)
  L_acc.append(list_acc)
  L_names.append(al_mtd + '_' + acq_fun)

  # num_iter = int(max_new_samples/batchsize)

  # al_mtd = 'local_max'
  # print(acq_fun, al_mtd)
  # start = timeit.default_timer() 
  # _, list_num_labels, list_acc = coreset_run_experiment(X, labels, W, coreset, num_iter=num_iter, method='Laplace',
  #                          display=False, use_prior=False, al_mtd=al_mtd, debug=False,
  #                          acq_fun=acq_fun, knn_data=knn_data, mtd_para=None,
  #                          savefig=False, savefig_folder='../BAL_figures', batchsize=batchsize,
  #                          dist_metric='angular', q=5, thresholding=0, randseed=0)
  # stop = timeit.default_timer()
  # L_time.append(stop - start)
  # L_num_labels.append(list_num_labels)
  # L_acc.append(list_acc)
  # L_names.append(al_mtd + '_' + acq_fun)

    
for i in range(len(L_acc)):
  plt.plot(L_num_labels[i], L_acc[i], label = L_names[i])

plt.legend()

title_str = 'Coreset size: ' + str(len(coreset)) + '; GL method: ' + method + '\n'
title_str += 'MSTAR CNNVAE: KNN: ' + str(knn_num) + '; Total num data: ' + str(X.shape[0])

plt.title(title_str)
plt.savefig(os.path.join('../BAL_figures', 'MSTAR_diff_acqfuns_' + al_mtd + '.png'))
plt.show()

### different distance methods

In [ ]:
distance_methods_list = ['dijkstra', 'poisson']
max_new_samples = 1000
batchsize=15

L_time = []
L_num_labels = []
L_acc = []
L_names = []

num_iter = int(max_new_samples/batchsize)

for distance_method in distance_methods_list:

  al_mtd = 'rs_kmeans'
  print(al_mtd, distance_method)
  mtd_para = (0.05, distance_method, 1.0, 0.1, 0.2, None, 200)
  start = timeit.default_timer() 
  _, list_num_labels, list_acc = bal.coreset_run_experiment(X, labels, W, coreset, num_iter=num_iter, method='Laplace',
                           display=False, use_prior=False, al_mtd=al_mtd, debug=False,
                           acq_fun=acq_fun, knn_data=knn_data, mtd_para=mtd_para,
                           savefig=False, savefig_folder='../BAL_figures', batchsize=batchsize,
                           dist_metric='angular', q=5, thresholding=0, randseed=0)
  stop = timeit.default_timer()
  L_time.append(stop - start)
  L_num_labels.append(list_num_labels)
  L_acc.append(list_acc)
  L_names.append(al_mtd + '_' + distance_method)
    
for i in range(len(L_acc)):
  plt.plot(L_num_labels[i], L_acc[i], label = L_names[i])

plt.legend()

title_str = 'Coreset size: ' + str(len(coreset)) + '; GL method: ' + method + '\n'
title_str += 'MSTAR CNNVAE: KNN: ' + str(knn_num) + '; Total num data: ' + str(X.shape[0])

plt.title(title_str)
plt.savefig(os.path.join('../BAL_figures', 'MSTAR_diff_distmtd_' + al_mtd + '.png'))
plt.show()

## OpenSarShip

In [ ]:
dataset_used = 'open_sar_ship'
_, labels = utils.load_dataset(dataset_used, return_torch=False, concatenate=True)
X, _ = utils.encode_pretrained('open_sar_ship', 'AlexNet', transformed=True)
print(X.shape)

In [ ]:
knn_num = 25
knn_data = gl.weightmatrix.knnsearch(X, 50, method='annoy', similarity='angular')
W = gl.weightmatrix.knn(X, knn_num, kernel = 'gaussian', knn_data=knn_data)
G = gl.graph(W)

In [ ]:
method = 'Laplace'
acq_fun = 'uc'
np.random.seed(25)
initial = gl.trainsets.generate(labels, rate=1).tolist()

In [ ]:
density_radius_param = .3
coreset = bal.coreset_dijkstras(G, rad = 0.25, DEBUGGING=False, data = X, initial=initial,
                                density_info = (True, density_radius_param, 1))
print("Coreset Size = {}\t Percent of data = {}%".format(len(coreset), round(100 * len(coreset) / len(X), 2)))
print("Coreset = ", coreset)

In [ ]:
acq_fun_list = ['uc', 'vopt', 'mc', 'mcvopt']
# acq_fun_list = ['mc', 'mcvopt']
max_new_samples = 600
batchsize=15

L_time = []
L_num_labels = []
L_acc = []
L_names = []

for acq_fun in acq_fun_list:

#     al_mtd = 'global_max'
#     print(acq_fun, al_mtd)
#     start = timeit.default_timer() 
#     _, list_num_labels, list_acc = bal.coreset_run_experiment(X, labels, W, coreset, num_iter=max_new_samples, method='Laplace',
#                             display=False, use_prior=False, al_mtd=al_mtd, debug=False,
#                             acq_fun=acq_fun, knn_data=knn_data, mtd_para=None,
#                             savefig=False, savefig_folder='../BAL_figures', batchsize=batchsize,
#                             dist_metric='angular', q=10, thresholding=0, randseed=0)
#     stop = timeit.default_timer()
#     L_time.append(stop - start)
#     L_num_labels.append(list_num_labels)
#     L_acc.append(list_acc)
#     L_names.append(acq_fun)

    num_iter = int(max_new_samples/batchsize)
    if acq_fun == 'mc':    
        num_iter += 14
    elif acq_fun == 'vopt' or acq_fun == 'mcvopt':
        num_iter += 1

    al_mtd = 'local_max'
    print(acq_fun, al_mtd)
    start = timeit.default_timer() 
    _, list_num_labels, list_acc = bal.coreset_run_experiment(X, labels, W, coreset, num_iter=num_iter, method='Laplace',
                            display=False, use_prior=False, al_mtd=al_mtd, debug=False,
                            acq_fun=acq_fun, knn_data=knn_data, mtd_para=None,
                            savefig=False, savefig_folder='../BAL_figures', batchsize=batchsize,
                            dist_metric='angular', q=10, thresholding=0, randseed=0)
    stop = timeit.default_timer()
    L_time.append(stop - start)
    L_num_labels.append(list_num_labels)
    L_acc.append(list_acc)
    L_names.append(acq_fun)
      
# for i in range(len(L_acc)):
#     plt.plot(L_num_labels[i], 100*L_acc[i], label=L_names[i])
# plt.plot(L_num_labels[0], 78.15 * np.ones_like(L_num_labels[0]), label='SoTA(CNN)', linestyle='--')
# plt.plot(L_num_labels[0], 76.99 * np.ones_like(L_num_labels[0]), label='TL (SLL CNN)', linestyle='--')
# plt.legend()
# plt.xlabel("Number of labeled points")
# plt.ylabel("Accuracy (%)")
# plt.ylim((None, 100))

# title_str = 'Coreset size: ' + str(len(coreset)) + '; GL method: ' + method + '; AL method: ' + 'sequential' + '\n'
# title_str += 'OpenSarShip: KNN: ' + str(knn_num) + '; Total num data: ' + str(X.shape[0])

# plt.title(title_str)
# plt.savefig(os.path.join('../BAL_figures', 'OpenSarShip_diff_acq_' + al_mtd + '.png'))
# plt.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twiny()

for i in range(len(L_acc)):
    ax1.plot(L_num_labels[i], 100*L_acc[i], label=L_names[i])
ax1.plot(L_num_labels[0], 78.15 * np.ones_like(L_num_labels[0]), label='SoTA', linestyle='--')
ax1.plot(L_num_labels[0], 76.99 * np.ones_like(L_num_labels[0]), label='TL', linestyle='--')
ax1.legend()
ax1.set_xlabel(r"Number of labeled points")
ax1.set_ylabel("Accuracy (%)")
ax1.set_ylim((50, 100))

title_str = 'Coreset size: ' + str(len(coreset)) + '; GL method: ' + method + '; AL method: ' + al_mtd + '\n'
title_str += 'OpenSarShip: KNN: ' + str(knn_num) + '; Total num data: ' + str(X.shape[0])

plt.title(title_str)

new_tick_locations = np.linspace(0.05, 0.25, 5) * 2296

def tick_function(X, n):
    V = X / n
    return ["%d%%" % (100*z) for z in V]

ax2.set_xlim(ax1.get_xlim())
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(new_tick_locations, X.shape[0]))
ax2.set_xlabel(r"Percentage of Training Data")
plt.savefig(os.path.join('../BAL_figures', 'OpenSarShip_diff_acq_' + al_mtd + '.png'), bbox_inches='tight')
plt.show()

## FUSAR

In [ ]:
dataset_used = 'fusar'
_, labels = utils.load_dataset(dataset_used, return_torch=False, concatenate=True)
X, _ = utils.encode_pretrained(dataset_used, 'ShuffleNet')
# encoded_data = encode_dataset(dataset_used, 'models/fusar_CNN.pt')
print(X.shape)

In [ ]:
knn_num = 25
knn_data = gl.weightmatrix.knnsearch(X, 50, method='annoy', similarity='angular')
W = gl.weightmatrix.knn(X, knn_num, kernel = 'gaussian', knn_data=knn_data)
G = gl.graph(W)

In [ ]:
method = 'Laplace'
acq_fun = 'uc'
np.random.seed(25)
initial = gl.trainsets.generate(labels, rate=1).tolist()

In [ ]:
# coreset = bal.plusplus_coreset(G, num_points=15, random_state=None, method='dijkstra', initial=initial)
density_radius_param = .25
coreset = bal.coreset_dijkstras(G, rad = 0.5, DEBUGGING=False, data = X, initial=initial,
                                density_info = (True, 0.5, 1))
print("Coreset Size = {}\t Percent of data = {}%".format(len(coreset), round(100 * len(coreset) / len(X), 2)))
print("Coreset = ", coreset)

In [ ]:
acq_fun_list = ['uc', 'vopt', 'mc', 'mcvopt']
# acq_fun_list = ['mc', 'mcvopt']
max_new_samples = 2500
batchsize=15

L_time = []
L_num_labels = []
L_acc = []
L_names = []

for acq_fun in acq_fun_list:

#     al_mtd = 'global_max'
#     print(acq_fun, al_mtd)
#     start = timeit.default_timer() 
#     _, list_num_labels, list_acc = bal.coreset_run_experiment(X, labels, W, coreset, num_iter=max_new_samples, method='Laplace',
#                             display=False, use_prior=False, al_mtd=al_mtd, debug=False,
#                             acq_fun=acq_fun, knn_data=knn_data, mtd_para=None,
#                             savefig=False, savefig_folder='../BAL_figures', batchsize=batchsize,
#                             dist_metric='angular', q=10, thresholding=0, randseed=0)
#     stop = timeit.default_timer()
#     L_time.append(stop - start)
#     L_num_labels.append(list_num_labels)
#     L_acc.append(list_acc)
#     L_names.append(acq_fun)

    num_iter = int(max_new_samples/batchsize)
#     if acq_fun == 'mc':    
#         num_iter += 14
#     elif acq_fun == 'vopt' or acq_fun == 'mcvopt':
#         num_iter += 1

    al_mtd = 'local_max'
    print(acq_fun, al_mtd)
    start = timeit.default_timer() 
    _, list_num_labels, list_acc = bal.coreset_run_experiment(X, labels, W, coreset, num_iter=num_iter, method='Laplace',
                            display=False, use_prior=False, al_mtd=al_mtd, debug=False,
                            acq_fun=acq_fun, knn_data=knn_data, mtd_para=None,
                            savefig=False, savefig_folder='../BAL_figures', batchsize=batchsize,
                            dist_metric='angular', q=10, thresholding=0, randseed=0)
    stop = timeit.default_timer()
    L_time.append(stop - start)
    L_num_labels.append(list_num_labels)
    L_acc.append(list_acc)
    L_names.append(acq_fun)
      
# for i in range(len(L_acc)):
#     plt.plot(L_num_labels[i], 100*L_acc[i], label=L_names[i])
# plt.plot(L_num_labels[0], 78.15 * np.ones_like(L_num_labels[0]), label='SoTA(CNN)', linestyle='--')
# plt.plot(L_num_labels[0], 76.99 * np.ones_like(L_num_labels[0]), label='TL (SLL CNN)', linestyle='--')
# plt.legend()
# plt.xlabel("Number of labeled points")
# plt.ylabel("Accuracy (%)")
# plt.ylim((None, 100))

# title_str = 'Coreset size: ' + str(len(coreset)) + '; GL method: ' + method + '; AL method: ' + 'sequential' + '\n'
# title_str += 'OpenSarShip: KNN: ' + str(knn_num) + '; Total num data: ' + str(X.shape[0])

# plt.title(title_str)
# plt.savefig(os.path.join('../BAL_figures', 'OpenSarShip_diff_acq_' + al_mtd + '.png'))
# plt.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twiny()

for i in range(len(L_acc)):
    ax1.plot(L_num_labels[i], 100*L_acc[i], label=L_names[i])
ax1.plot(L_num_labels[0], 86.69 * np.ones_like(L_num_labels[0]), label='SoTA', linestyle='--')
ax1.plot(L_num_labels[0], 75.36 * np.ones_like(L_num_labels[0]), label='TL', linestyle='--')
ax1.legend()
ax1.set_xlabel(r"Number of labeled points")
ax1.set_ylabel("Accuracy (%)")
ax1.set_ylim((40, 100))

title_str = 'Coreset size: ' + str(len(coreset)) + '; GL method: ' + method + '; AL method: ' + al_mtd + '\n'
title_str += 'FuSar: KNN: ' + str(knn_num) + '; Total num data: ' + str(X.shape[0])

plt.title(title_str)

new_tick_locations = np.linspace(0.05, 0.5, 10) * X.shape[0]

def tick_function(X, n):
    V = X / n
    return ["%d%%" % (100*z) for z in V]

ax2.set_xlim(ax1.get_xlim())
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(new_tick_locations, X.shape[0]))
ax2.set_xlabel(r"Percentage of Training Data")
plt.savefig(os.path.join('../BAL_figures', 'FuSar_diff_acq_' + al_mtd + '.png'), bbox_inches='tight')
plt.show()

In [ ]:
# acq_fun_list = ['uc', 'vopt', 'mc', 'mcvopt']
acq_fun_list = ['mc', 'mcvopt']
max_new_samples = 2500
batchsize=15

for acq_fun in acq_fun_list:
    L_time = []
    L_num_labels = []
    L_acc = []
    L_names = []

    al_mtd = 'global_max'
    print(acq_fun, al_mtd)
    start = timeit.default_timer() 
    _, list_num_labels, list_acc = bal.coreset_run_experiment(X, labels, W, coreset, num_iter=max_new_samples, method='Laplace',
                            display=False, use_prior=False, al_mtd=al_mtd, debug=False,
                            acq_fun=acq_fun, knn_data=knn_data, mtd_para=None,
                            savefig=False, savefig_folder='../BAL_figures', batchsize=batchsize,
                            dist_metric='angular', q=10, thresholding=0, randseed=0)
    stop = timeit.default_timer()
    L_time.append(stop - start)
    L_num_labels.append(list_num_labels)
    L_acc.append(list_acc)
    L_names.append(al_mtd)

    num_iter = int(max_new_samples/batchsize)

    al_mtd = 'local_max'
    print(acq_fun, al_mtd)
    start = timeit.default_timer() 
    _, list_num_labels, list_acc = bal.coreset_run_experiment(X, labels, W, coreset, num_iter=num_iter, method='Laplace',
                            display=False, use_prior=False, al_mtd=al_mtd, debug=False,
                            acq_fun=acq_fun, knn_data=knn_data, mtd_para=None,
                            savefig=False, savefig_folder='../BAL_figures', batchsize=batchsize,
                            dist_metric='angular', q=10, thresholding=0, randseed=0)
    stop = timeit.default_timer()
    L_time.append(stop - start)
    L_num_labels.append(list_num_labels)
    L_acc.append(list_acc)
    L_names.append(al_mtd)

    al_mtd = 'rs_kmeans'
    print(acq_fun, al_mtd)
    mtd_para = (0.05, 'dijkstra', 1.0, 0.1, 0.2, None, 200, False)
    start = timeit.default_timer() 
    _, list_num_labels, list_acc = bal.coreset_run_experiment(X, labels, W, coreset, num_iter=num_iter, method='Laplace',
                            display=False, use_prior=False, al_mtd=al_mtd, debug=False,
                            acq_fun=acq_fun, knn_data=knn_data, mtd_para=mtd_para,
                            savefig=False, savefig_folder='../BAL_figures', batchsize=batchsize,
                            dist_metric='angular', q=10, thresholding=0, randseed=0)
    stop = timeit.default_timer()
    L_time.append(stop - start)
    L_num_labels.append(list_num_labels)
    L_acc.append(list_acc)
    L_names.append(al_mtd)

    al_mtd = 'acq_sample'
    print(acq_fun, al_mtd)
    start = timeit.default_timer() 
    _, list_num_labels, list_acc = bal.coreset_run_experiment(X, labels, W, coreset, num_iter=num_iter, method='Laplace',
                            display=False, use_prior=False, al_mtd=al_mtd, debug=False,
                            acq_fun=acq_fun, knn_data=knn_data, mtd_para=None,
                            savefig=False, savefig_folder='../BAL_figures', batchsize=batchsize,
                            dist_metric='angular', q=10, thresholding=0, randseed=0)
    stop = timeit.default_timer()
    L_time.append(stop - start)
    L_num_labels.append(list_num_labels)
    L_acc.append(list_acc)
    L_names.append(al_mtd)

    al_mtd = 'rs_kmeans'
    print(acq_fun, al_mtd)
    cw = lambda x,y: ( x / (x + y) ) ** 10
    mtd_para = (0.05, 'dijkstra', 1.0, 0.1, 0.2, cw, 200, False)
    start = timeit.default_timer() 
    _, list_num_labels, list_acc = bal.coreset_run_experiment(X, labels, W, coreset, num_iter=num_iter, method='Laplace',
                            display=False, use_prior=False, al_mtd=al_mtd, debug=False,
                            acq_fun=acq_fun, knn_data=knn_data, mtd_para=None,
                            savefig=False, savefig_folder='../BAL_figures', batchsize=batchsize,
                            dist_metric='angular', q=1, thresholding=0, randseed=0)
    stop = timeit.default_timer()
    L_time.append(stop - start)
    L_num_labels.append(list_num_labels)
    L_acc.append(list_acc)
    L_names.append(al_mtd + '_c')
      
    for i in range(len(L_acc)):
        plt.plot(L_num_labels[i], 100*L_acc[i], label=L_names[i])
    plt.plot(L_num_labels[0], 86.69 * np.ones_like(L_num_labels[0]), label='SoTA(CNN)', linestyle='--')
    plt.plot(L_num_labels[0], 75.36 * np.ones_like(L_num_labels[0]), label='TL (SLL CNN)', linestyle='--')
    plt.legend()
    plt.xlabel("Number of labeled points")
    plt.ylabel("Accuracy (%)")
    plt.ylim((None, 100))

    title_str = 'Coreset size: ' + str(len(coreset)) + '; GL method: ' + method + '; Al function: ' + acq_fun + '\n'
    title_str += 'FUSAR: KNN: ' + str(knn_num) + '; Total num data: ' + str(X.shape[0])

    plt.title(title_str)
    plt.savefig(os.path.join('../BAL_figures', 'FUSAR_'+acq_fun+'_diff_mtd.png'))
    plt.show()